In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
import pandas as pd
import os
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn.objects as so 
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import BinaryCrossentropy
tf.config.run_functions_eagerly(True)

#custom imports
from src.processors.Preprocessor import Preprocessor


2023-04-13 11:23:32.677986: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
PATH = r'/Users/michaelschaid/GitHub/dopamine_modeling/data/gaby_test/aggregated_data.parquet.gzp'
PATH_TO_SAVE = '/Users/michaelschaid/GitHub/dopamine_modeling/data/gaby_test/processed_data.parquet.gzp'

# Initial preprocessing of data:
* load data
* hot encode categorical variables
* select features and target variable
* split data into train and test sets
  * started withn 80/20 split

In [4]:
preprocessor  = Preprocessor(PATH)
preprocessor.load_data()
preprocessor.one_hot_encode(*['event', 'sensor'])
preprocessor.save_features(PATH_TO_SAVE)


features = ['day', 'time', 'trial', 
            'signal','event_cue', 'event_shock', 
            'sensor_D1', 'sensor_D2', 'sensor_DA']
target = 'signal'

preprocessor.split_train_test(X_labels = features, y_label=target)

### Standard fully connected 

In [ ]:
training_sensor_DA = preprocessor.dummy_data.query('day < 5')
testing_sensor_DA = preprocessor.dummy_data.query('trial >= 7')

X_train = training_sensor_DA[features]
y_train = training_sensor_DA[target]

X_test = testing_sensor_DA[features]
y_test = testing_sensor_DA[target]


In [ ]:
sample = preprocessor.dummy_data.head(100)
X_train, X_test, y_train, y_test = train_test_split(sample[features], sample[target], test_size=0.2, random_state=42)
sample.to_parquet('/Users/michaelschaid/GitHub/dopamine_modeling/data/gaby_test/testing_data.parquet.gzp')

In [ ]:
model = Sequential([
    Dense(units = 128, activation='sigmoid'),
    Dense(units = 25, activation = 'sigmoid'),
    Dense(units = 1, activation = 'sigmoid')])
model.compile(loss = BinaryCrossentropy())
model.fit(X_train, y_train, epochs = 100)


In [9]:
model.layers

In [11]:
model.evaluate(X_test, y_test, verbose=2)

1/1 - 0s - loss: -3.1561e+00 - 36ms/epoch - 36ms/step


/Users/michaelschaid/opt/anaconda3/envs/dope/lib/python3.11/site-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
/Users/michaelschaid/opt/anaconda3/envs/dope/lib/python3.11/site-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


-3.1560609340667725

In [13]:
model.get_metrics_result()

{'loss': <tf.Tensor: shape=(), dtype=float32, numpy=-3.156061>}

In [16]:

save_model_path = os.path.join('/Users/michaelschaid/GitHub/dopamine_modeling/results/models','saved_model_2')
model.save(save_model_path)

INFO:tensorflow:Assets written to: /Users/michaelschaid/GitHub/dopamine_modeling/results/models/saved_model_2/assets


INFO:tensorflow:Assets written to: /Users/michaelschaid/GitHub/dopamine_modeling/results/models/saved_model_2/assets
